In [104]:
import sys
import os
import pandas as pd
from datetime import datetime
import librosa_trans
import re
from IPython.display import clear_output, display
import numpy as np

In [174]:
sound_path = os.path.abspath('/media/yiying/TOS/')
test_list = ['normal','queenless']

data=pd.DataFrame()

for test in test_list:
    dir_path = sorted(os.listdir(os.path.join(sound_path, test)))
    for file in dir_path:
        time_detail = re.search( r'(.*).wav', file).group(1)
        date,time = time_detail.split('-')
        date = date.split('_')[-1]
        time_split = datetime.strptime(date+time,'%Y%m%d%H%M%S')
        data=data.append({'date':time_split,'path':os.path.join(sound_path,test,file),'id':'Yilan_queenless4','district':'','state':test}, ignore_index=True)
        
data.to_csv('/media/yiying/TOS/datalist.csv',index_label=None)

In [175]:
# Load the metadata from the generated CSV
metadata = pd.read_csv('/media/yiying/TOS/datalist.csv')

# Examine dataframe
print("Metadata length:", len(metadata))
metadata.tail()

Metadata length: 319


,Unnamed: 0,date,district,id,path,state
314,314,2021-09-04 17:16:18,east,Yilan_queenless4,/media/yiying/TOS/queenless/20210904-171618.wav,queenless
315,315,2021-09-04 17:17:28,east,Yilan_queenless4,/media/yiying/TOS/queenless/20210904-171728.wav,queenless
316,316,2021-09-04 17:18:38,east,Yilan_queenless4,/media/yiying/TOS/queenless/20210904-171838.wav,queenless
317,317,2021-09-04 17:19:48,east,Yilan_queenless4,/media/yiying/TOS/queenless/20210904-171948.wav,queenless
318,318,2021-09-04 17:20:58,east,Yilan_queenless4,/media/yiying/TOS/queenless/20210904-172058.wav,queenless


In [176]:


features = []
labels = []
colonies = []
frames_max = 0
counter = 0
total_samples = len(metadata)
n_mfcc = 40

# transform to MFCC
for index, row in metadata.iterrows():


    file_path = os.path.abspath(str(row["path"]))
    colony_label = row["id"]
    state_label = row["state"]

    # Extract MFCCs (do not add padding)
    mfccs = librosa_trans.mfcc_transform(file_path)

    # Save current frame count
    num_frames = mfccs.shape[1]

    # Add row (feature / label)
    features.append(mfccs)
    labels.append(state_label)
    colonies.append(colony_label)

    # Update frames maximum
    if (num_frames > frames_max):
        frames_max = num_frames

    clear_output(wait=True)
    print("Progress: {}/{}".format(index,total_samples))
    print("Last file: ", file_path)


# padding to max_length

padded = []

# Add padding
mfcc_max_padding = frames_max
for j in range(len(features)):
    size = len(features[j][0])
    if (size < mfcc_max_padding):
        pad_width = mfcc_max_padding - size
        px = np.pad(features[j], 
                    pad_width=((0, 0), (0, pad_width)), 
                    mode='constant', 
                    constant_values=(0,))
    else:
        px = features[j]

    padded.append(px)


# save file

X = np.array(padded)
y = np.array(labels)

# Optionally save the features to disk
np.save(f"./data/Yilan-q-4-X-mfcc-augmented", X)
np.save(f"./data/Yilan-q-4-y-mfcc-augmented", y)


Progress: 318/319
Last file:  /media/yiying/TOS/queenless/20210904-172058.wav


In [94]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sn
import matplotlib.pyplot as plt
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
# from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split


print(tf.__version__)
# print(keras.__version__)

2.5.0


In [407]:
def concat_dataset():
    path = './data/'
    types = ['normal', 'imd']
    features = labels = np.array([])
    

                
    features_filename = f'Yilan-imd1-X-mfcc-augmented.npy'
    labels_filename = f'Yilan-imd1-y-mfcc-augmented.npy'

    curr_features = np.load(os.path.join(path, features_filename))
    curr_labels = np.load(os.path.join(path, labels_filename))

    if features.size == 0:
        features = curr_features
    else:
        features = np.concatenate((features, curr_features))

    if labels.size == 0:
        labels = curr_labels
    else:
        labels = np.concatenate((labels, curr_labels))
    
    return (features, labels)

        
dataset = concat_dataset()
print(dataset[0])
print(dataset[1])

[[[-1.0000000e+00 -1.0000000e+00 -1.0000000e+00 ... -1.0000000e+00
   -1.0000000e+00 -1.0000000e+00]
  [ 3.2259524e-01  3.3138469e-01  3.3090383e-01 ...  3.4143329e-01
    3.3608040e-01  3.2647151e-01]
  [ 1.3556598e-01  1.3633381e-01  1.4121857e-01 ...  1.3879907e-01
    1.3399698e-01  1.3289391e-01]
  ...
  [-1.1801712e-02 -9.5799053e-03 -1.1268334e-02 ... -1.5293019e-02
   -9.2151053e-03 -5.7651279e-03]
  [ 1.1316449e-03  7.4460852e-04 -7.9328669e-03 ... -9.6429177e-03
    1.6636715e-03  1.2104601e-02]
  [-9.9100508e-03 -7.7228970e-04  3.1809206e-03 ...  1.7828937e-03
    7.6698898e-03  1.3510254e-02]]

 [[-1.0000000e+00 -1.0000000e+00 -1.0000000e+00 ... -1.0000000e+00
   -1.0000000e+00 -1.0000000e+00]
  [ 3.0290309e-01  3.1254244e-01  3.2148051e-01 ...  3.2798216e-01
    3.2205033e-01  3.1877950e-01]
  [ 1.4110005e-01  1.4812668e-01  1.4018364e-01 ...  1.4237869e-01
    1.4523640e-01  1.4995725e-01]
  ...
  [-9.3802046e-03 -8.4800785e-03 -1.3300611e-02 ... -6.4719091e-03
   -7.6948

In [408]:
def transfer(x):
    if x=='normal':
        return 2
    if x=='flu':
        return 0
    if x=='imd':
        return 1
    if x=='queenless':
        return 3


features = dataset[0]
labels = np.fromiter(map(transfer, dataset[1]), dtype=np.int) 
print(labels)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [412]:
from tensorflow import keras
model = keras.models.load_model('./model/multiclass_model_q_yunlinankeng.h5')
print("Evaluate on test data")
results = model.evaluate(features, labels)#, batch_size=128
print("test loss, test acc:", results)

Evaluate on test data
17/17 [==============================] - 1s 6ms/step - loss: 5.2875 - accuracy: 0.0000e+00
test loss, test acc: [5.287547588348389, 0.0]


In [413]:
predictions = model.predict(features)
classes = np.argmax(predictions, axis=1)
print(classes)

[0 0 0 0 0 0 3 0 3 0 3 3 0 3 0 0 0 0 3 0 0 3 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0
 3 3 0 0 3 3 3 3 3 3 0 3 0 0 3 0 3 3 0 3 0 3 0 0 3 3 3 3 3 3 3 3 3 0 3 3 3
 0 0 3 3 0 3 0 0 3 3 0 3 3 0 0 3 0 3 0 3 3 3 3 0 0 0 0 0 3 0 0 0 0 0 3 3 0
 0 0 3 3 0 0 0 0 0 3 0 3 0 3 0 3 0 0 0 3 3 3 3 0 0 0 3 3 0 0 0 0 0 0 3 0 0
 0 0 3 3 0 0 0 0 3 0 0 3 0 3 0 0 0 0 0 3 0 3 3 0 0 3 0 0 0 0 3 3 0 0 3 3 3
 0 3 3 0 0 0 0 0 0 0 3 0 0 3 3 3 3 3 3 0 0 0 0 0 0 3 0 0 0 0 0 0 3 0 0 0 0
 3 3 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 3 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 0 3 3 3
 3 3 3 3 3 0 0 0 0 0 0 3 0 0 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 2 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 3 2 2 2 3 2 2 2 2 3 3 2 3
 3 3 3 2 2 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 2 3 3 3 2 2 3 